In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

import sys
sys.path.append('../utils')
sys.path.append('../dFC')
import widefield_utils
import connectivity_measures

In [ ]:
mask = np.load(r"/home/k21208334/calcium_analyses/data/dowsampled_tight_mask.npy")
good_indices = np.ravel(mask)

In [ ]:
h=300
w=304
H=50 # this is the size of downsampling. We have to find a better way to do this
W=51
start = 0
end = 25000

names = ["NXAK22.1A","NXAK14.1A","NXAK7.1B","NXAK4.1B","NRXN78.1D","NRXN78.1A"]
names_knock = ["NXAK24.1C","NXAK20.1B","NXAK16.1B","NXAK10.1A","NXAK4.1A","NRXN71.2A"]

n_comp = 10 # i choose this n of components because it's the most convincing

In [ ]:
signals = np.load('/home/k21208334/calcium_analyses/data/ICA/signals_n=' + str(n_comp) + '.npy')
comps = np.load('/home/k21208334/calcium_analyses/data/ICA/components_n=' + str(n_comp) + '.npy')

In [ ]:
# For each component, I compare each ICA component absolute amplitude and I do walking vs non walking

In [ ]:
# create walking timeseries note the cool python command I use:
walking_wild = np.concatenate([np.load('/home/k21208334/calcium_analyses/data/walking/'+name+'.npy') for name in names])
walking_knock = np.concatenate([np.load('/home/k21208334/calcium_analyses/data/walking/'+name+'.npy') for name in names_knock])


In [ ]:
len = signals.shape[0]

for i in range(n_comp):
    fig = connectivity_measures.show_array(comps[:,i],H,W,good_indices,comps[:,i].min(),comps[:,i].max())
    fig.set_size_inches(2,2)